TRYING TO LOCATE THE FACE DIRECTLY IN 1 STEP
ALL RELEVANT FILES ARE IN HAAR_TRAIN_1STEP

Cascade face detection:
1) Label images
    1a) label the head
    1b) label non-head areas
2) Train the classifier
    2a) prepare negative samples
    2b) prepare positive samples
    2c) cascade training
3) Test the classifier
    what is a good image size, min_neighbor value, minhitrate, etc.
    
useful links:

http://docs.opencv.org/2.4/doc/user_guide/ug_traincascade.html#cascade-training
http://docs.opencv.org/2.4/modules/objdetect/doc/cascade_classification.html?highlight=detectmultiscale
http://www.bogotobogo.com/python/OpenCV_Python/python_opencv3_Image_Object_Detection_Face_Detection_Haar_Cascade_Classifiers.php
https://sites.google.com/site/5kk73gpu2012/assignment/viola-jones-face-detection#TOC-Image-Pyramid


Label whale heads

In [3]:
import os
from random import choice
from PIL import Image
import numpy as np
import pickle
import pylab as pl

files = []

for file in os.listdir("../imgs"):
    if file.endswith(".jpg"):
        files.append(file[:-4])

# number of images to select
n = 100

# select one file, check whether it's already in clicks, detect the nose and the blowhole,
# cut the face out, then save the clicks
i = 0
while i < n:  
    file_to_analyze = choice(files)
    # check whether this file is already in clicks
    already_analized_files = []
    for file in os.listdir("../haar_train_1step/clicks_head"):
        if file.endswith(".save"):
            already_analized_files.append(file[:-5])
    # stupidity check
    if len(already_analized_files) == len(files):
        # all files have been analyzed.
        break
    # if file is not in directory already, mark the nose and blowhole    
    if (file_to_analyze not in already_analized_files):
        print file_to_analyze+'.jpg'
        im = np.array(Image.open('../imgs/'+file_to_analyze+'.jpg'))
        
        pl.figure(figsize = (15,9))  
        pl.imshow(im,aspect='equal')
        # coarse selection of the head - first select two points to mark a rectangle around the head 
        # be generous and don't mark points too close to the whale's face
        rect = pl.ginput(2,timeout=0)
        pl.close()
        
        # the rectangle is cut 
        if rect[0][0] != rect[1][0] or rect[0][1] != rect[1][1]:
            # the x coordinate is the second coordinate [1] on display
            xmin = np.min([rect[0][1],rect[1][1]])
            xmax = np.max([rect[0][1],rect[1][1]])
            ymin = np.min([rect[0][0],rect[1][0]])
            ymax = np.max([rect[0][0],rect[1][0]])
        else:
            print 'rectangle was not cut out properly'
            raise ValueError
             
        im_cut = im[xmin:xmax,ymin:ymax]

        # display the magnified rectangle to mark the exact location of the nose and the blowhole
        pl.figure(figsize = (15,9))
        pl.imshow(im_cut,aspect='equal',interpolation='nearest')
        # step 1: mark the top of the nose
        # step 2: mark the middle of the blowhole
        head = pl.ginput(2,timeout=0)
        
        pl.close()
        
        # dump clicks
        f = open('../haar_train_1step/clicks_head/'+file_to_analyze+'.save','w')
        pickle.dump([file_to_analyze,rect,head],f)
        f.close()
        
        i = i + 1   


w_3766.jpg
w_7631.jpg
w_5921.jpg
w_5072.jpg
w_6899.jpg
w_2049.jpg
w_9959.jpg
w_7938.jpg
w_3572.jpg
w_746.jpg
w_2381.jpg
w_4762.jpg
w_9489.jpg
w_882.jpg
w_8511.jpg
w_5857.jpg
w_7884.jpg
w_5962.jpg
w_2892.jpg
w_8383.jpg
w_8412.jpg
w_10878.jpg
w_9931.jpg
w_5104.jpg
w_5288.jpg
w_8966.jpg
w_8640.jpg
w_1723.jpg
w_3533.jpg
w_2614.jpg
w_5367.jpg
w_9219.jpg
w_1012.jpg
w_8100.jpg
w_1884.jpg
w_4310.jpg
w_7109.jpg
w_4752.jpg
w_107.jpg
w_7562.jpg
w_4697.jpg
w_1816.jpg
w_5650.jpg
w_10860.jpg
w_3258.jpg
w_432.jpg
w_11239.jpg
w_8662.jpg
w_1978.jpg
w_4403.jpg
w_2037.jpg
w_3829.jpg
w_11337.jpg
w_6118.jpg
w_4978.jpg
w_5200.jpg
w_40.jpg
w_6176.jpg
w_11142.jpg
w_6574.jpg
w_10978.jpg
w_6398.jpg
w_10925.jpg
w_6152.jpg
w_6332.jpg
w_9617.jpg
w_10575.jpg
w_1689.jpg
w_7500.jpg
w_10116.jpg
w_3923.jpg
w_1481.jpg
w_11356.jpg
w_2472.jpg
w_7744.jpg
w_11366.jpg
w_10875.jpg
w_6807.jpg
w_3512.jpg
w_1075.jpg
w_1048.jpg
w_7019.jpg
w_6984.jpg
w_5437.jpg
w_6482.jpg
w_7262.jpg
w_9950.jpg
w_6081.jpg
w_6192.jpg
w_10346.jpg
w_8

Label negative areas, areas that do not contain a whale's head

In [4]:
import os
from random import choice
from PIL import Image
import numpy as np
import pickle
import pylab as pl

files = []

# images with heads already labeled
for file in os.listdir("../haar_train_1step/clicks_head"):
    if file.endswith(".save"):
        files.append(file[:-5])

# check which of these files were analized already
already_analized_files = []
for file in os.listdir("../haar_train_1step/clicks_other"):
    if file.endswith(".save"):
        already_analized_files.append(file[:-5])

# click on the rest
files_to_analyze = list(set(files) - set(already_analized_files))

for i in files_to_analyze: 
    print i
    im = np.array(Image.open('../imgs/'+i+'.jpg'))

    pl.figure(figsize = (14,8))  
    pl.imshow(im,aspect='equal')
    # select a rectangle without a face
    rect = pl.ginput(2,timeout=0)
    pl.close()

    # dump clicks
    f = open('../haar_train_1step/clicks_other/'+i+'.save','w')
    pickle.dump([i,rect],f)
    f.close()


w_6899
w_5288
w_9931
w_7262
w_5921
w_1816
w_6152
w_1481
w_7019
w_4697
w_11366
w_3554
w_8412
w_8640
w_2381
w_1689
w_10978
w_4050
w_10878
w_4762
w_1048
w_10875
w_6332
w_1884
w_5200
w_10575
w_10860
w_7926
w_5437
w_6574
w_4752
w_1978
w_3258
w_4978
w_9950
w_9617
w_40
w_7631
w_9959
w_11248
w_6192
w_3533
w_7938
w_8966
w_7500
w_2037
w_7744
w_5650
w_5367
w_11239
w_746
w_1723
w_1012
w_5966
w_5962
w_7109
w_10925
w_6118
w_8383
w_107
w_2049
w_8662
w_432
w_2892
w_5072
w_8511
w_7562
w_6482
w_4310
w_6398
w_7884
w_4403
w_11337
w_11142
w_2614
w_9489
w_11356
w_5104
w_9219
w_6081
w_3829
w_10538
w_882
w_1075
w_6176
w_3512
w_6984
w_3572
w_10116
w_10346
w_8106
w_8100
w_6807
w_492
w_5857
w_3923
w_6167
w_3688
w_2472
w_3766


Prepare negative images

In [8]:
import pickle
import os
import glob
from scipy.ndimage.interpolation import rotate
from scipy.misc import imresize
from scipy.misc import imsave
from PIL import Image
import numpy as np

# delete all images in directory
files = glob.glob('../haar_train_1step/negative_sample/*')
for f in files:
    os.remove(f)

non_whale_faces = []
for file in os.listdir("../haar_train_1step/clicks_other"):
    if file.endswith(".save"):
        non_whale_faces.append(file[:-5])

# calculate average mean and std of images
#mean = 0e0
#std = 0e0
#for i in non_whale_faces:
#    im = np.array(Image.open('../imgs/'+file_to_analyze+'.jpg').convert('L'))
#    mean = mean + np.mean(im)
#    std = std + np.std(im)
#mean = mean / float(len(non_whale_faces))
#std = std / float(len(non_whale_faces))
#print mean, std

mean = 111.031397463 
std = 18.4898562094


for i in non_whale_faces:
    # pickle load
    f = open('../haar_train_1step/clicks_other/'+i+'.save','r')
    [file_to_analyze,rect] = pickle.load(f)
    f.close()
    
    # open image
    im = np.array(Image.open('../imgs/'+file_to_analyze+'.jpg').convert('L'))
    
    # cut out the rectangle
    if rect[0][0] != rect[1][0] or rect[0][1] != rect[1][1]:
        # the x coordinate is the second coordinate [1] on display
        xmin = np.min([rect[0][1],rect[1][1]])
        xmax = np.max([rect[0][1],rect[1][1]])
        ymin = np.min([rect[0][0],rect[1][0]])
        ymax = np.max([rect[0][0],rect[1][0]])
    else:
        print 'rectangle was not cut out properly'
        raise ValueError
    
    im_cut = im[xmin:xmax,ymin:ymax]
    
    # standardize the background pics using whole pic values
    im_cut = (im_cut - np.mean(im_cut)) / np.std(im_cut)
    # restandardize based on the average means and stds
    im_cut = (im_cut*std)+mean
    im_cut[im_cut < 0e0] = 0e0
    im_cut[im_cut > 255e0] = 255e0
    
    imsave('../haar_train_1step/negative_sample/'+i+'_0.jpg', im_cut)
    # rotate images
    imsave('../haar_train_1step/negative_sample/'+i+'_1.jpg', np.rot90(im_cut,k=1))
    imsave('../haar_train_1step/negative_sample/'+i+'_2.jpg', np.rot90(im_cut,k=2))
    imsave('../haar_train_1step/negative_sample/'+i+'_3.jpg', np.rot90(im_cut,k=3))

# create the description file of negative samples
# these files are in ../haar_train/negative_sample
non_whale_faces = []
for file in os.listdir("../haar_train_1step/negative_sample"):
    if file.endswith(".jpg"):
        non_whale_faces.append(file)
f = open('../haar_train_1step/bg.txt','w')
for i in non_whale_faces:
    f.write('../haar_train_1step/negative_sample/'+i+'\n')
f.close()

Prepare positive images, step 1

In [9]:
import pickle
import os
import glob
from scipy.ndimage.interpolation import rotate
from scipy.misc import imresize
from scipy.misc import imsave
from PIL import Image
import numpy as np

## parameters for the whale face images:
# edge width in unit of nose-blowhole distance
edge = 0.4
# color or grayscale image
color = True

# delete all images in directory
files = glob.glob('../haar_train_1step/clicked_whale_faces/*')
for f in files:
    os.remove(f)

whale_faces = []
for file in os.listdir("../haar_train_1step/clicks_head"):
    if file.endswith(".save"):
        whale_faces.append(file[:-5])

for i in whale_faces:
    # pickle load
    f = open('../haar_train_1step/clicks_head/'+i+'.save','r')
    [file_to_analyze,rect,head] = pickle.load(f)
    f.close()
    
    # open image
    im = np.array(Image.open('../imgs/'+i+'.jpg'))
    
    # cut out the rectangle
    if rect[0][0] != rect[1][0] or rect[0][1] != rect[1][1]:
        # the x coordinate is the second coordinate [1] on display
        xmin = np.min([rect[0][1],rect[1][1]])
        xmax = np.max([rect[0][1],rect[1][1]])
        ymin = np.min([rect[0][0],rect[1][0]])
        ymax = np.max([rect[0][0],rect[1][0]])
    else:
        print 'rectangle was not cut out properly'
        raise ValueError
    
    # head clicks in im coordinates
    headc = np.zeros([2,2])
    headc[0,1] = head[0][1] + xmin
    headc[1,1] = head[1][1] + xmin
    headc[0,0] = head[0][0] + ymin
    headc[1,0] = head[1][0] + ymin
    
    # rotate the image such that the nose-blowhole line is vertical, nose is up
    dx = headc[0,1] - headc[1,1]
    dy = headc[0,0] - headc[1,0]
    theta = np.arctan2(dx,dy) + np.pi/2e0 # added 90 deg to rotate the nose up
    if dx/dy >= 0:
        theta = theta + 2e0*np.pi # range is between 0 and 2pi now
    im_rotate = rotate(im,np.degrees(theta),mode='nearest')

    # rotate the head and blowhole coordinates
    # the center of im_cut
    xc_cut = np.shape(im)[0]/2e0 + 0.5
    yc_cut = np.shape(im)[1]/2e0 + 0.5
    # the center of im_rotate
    xc_rot = np.shape(im_rotate)[0]/2e0 + 0.5
    yc_rot = np.shape(im_rotate)[1]/2e0 + 0.5
    # rotate   
    nose = np.zeros(2)
    nose[1] = (headc[0,1]-xc_cut)*np.cos(theta) - (headc[0,0]-yc_cut)*np.sin(theta) + xc_rot
    nose[0] = (headc[0,1]-xc_cut)*np.sin(theta) + (headc[0,0]-yc_cut)*np.cos(theta) + yc_rot
    blowhole = np.zeros(2)
    blowhole[1] = (headc[1,1]-xc_cut)*np.cos(theta) - (headc[1,0]-yc_cut)*np.sin(theta) + xc_rot
    blowhole[0] = (headc[1,1]-xc_cut)*np.sin(theta) + (headc[1,0]-yc_cut)*np.cos(theta) + yc_rot

    # cut a square piece out and add edge*distance edge around the selected region
    distance = np.sqrt((nose[0]-blowhole[0])**2e0 + (nose[1]-blowhole[1])**2e0)
    im_save = im_rotate[nose[1]-distance*edge:blowhole[1]+distance*edge,nose[0]-distance*(0.5+edge):nose[0]+distance*(0.5+edge)]
        
    # save
    if color:
        imsave('../haar_train_1step/clicked_whale_faces/'+i+'.jpg', im_save)
    else:
        im_grayscale = np.average(imresize(im_save,[im_size,im_size]),axis=2, weights=[0.299, 0.587, 0.144])
        imsave('../haar_train_1step/clicked_whale_faces/'+i+'.jpg', im_grayscale)
        

-c:57: RuntimeWarning: divide by zero encountered in double_scalars


Prepare positive images, step 2

In [10]:
import os
import glob
from PIL import Image
from scipy.ndimage.interpolation import rotate
import numpy as np
from random import choice
from scipy.misc import imsave

# based on previous cell
mean = 111.031397463 
std = 18.4898562094

# number of random angles for rotation
n = 10

whale_faces = []
for file in os.listdir("../haar_train_1step/clicked_whale_faces"):
    if file.endswith(".jpg"):
        whale_faces.append(file)

# delete all images in directory
files = glob.glob('../haar_train_1step/positive_sample_normalized_straight/*')
for f in files:
    os.remove(f)        

f = open('../haar_train_1step/annotations_normalized_straight.lst','w')
ii = 0
for j in range(n):
    for i in whale_faces:
        # open file
        face = np.array(Image.open('../haar_train_1step/clicked_whale_faces/'+i).convert('L'))
        
        # rotate with a random angle and cut the central part out
        angle = np.random.uniform(-5,5)        
        face_rot = rotate(face,angle)
        (x,y) = np.shape(face)
        x = int(x/np.sqrt(2e0))
        (x_rot,y_rot) = np.shape(face_rot)
        face_rotate = np.zeros([x,x])
        face_rotate = face_rot[x_rot/2e0 - x/2e0:x_rot/2e0 + x/2e0,x_rot/2e0 - x/4e0:x_rot/2e0 + x/4e0]        
        
        # standardize the face using the whole image values
        face_rotate = (face_rotate - np.mean(face_rotate)) / np.std(face_rotate)
        # restandardize based on the average means and stds
        face_rotate = (face_rotate*std)+mean
        face_rotate[face_rotate < 0e0] = 0e0
        face_rotate[face_rotate > 255e0] = 255e0
        
        # save image
        imsave('../haar_train_1step/positive_sample_normalized_straight/'+str(ii)+'.jpg', face_rotate)
        # write annotations.lst
        f.write('positive_sample_normalized_straight/'+str(ii)+'.jpg 1 0 0 '+str(x)+' '+str(x)+'\n')
        
        ii = ii + 1
f.close()    



Run opencv: create vec file, and train the cascade

In [13]:
import os
import glob
import subprocess
import random
import sys

# size of positive images
size = 80


# create vec file from annotated files
out = subprocess.check_output(["opencv_createsamples","-vec", "../haar_train_1step/whale_face_"+str(size)+"_normalized_straight.vec", "-info", 
                               "../haar_train_1step/annotations_normalized_straight.lst","-num", "10000", "-w", str(size/2), "-h", str(size)])

print out

# an example command line to copy into a terminal
# opencv_traincascade -data ../haar_train/cascade -vec ../haar_train/whale_face.vec -bg ../haar_train/bg.txt 
# -numStages 20 -minHitRate 0.999 -maxFalseAlarmRate 0.5 -numPos 5000 -numNeg 800 -w 24 -h 24 -mode ALL 
# -precalcValBufSize 1024 -precalcIdxBufSize 1024
# see http://stackoverflow.com/questions/10863560/haar-training-opencv-assertion-failed why numPos is not 5000
#with open('test.log', 'w') as f:
#    process = subprocess.Popen(["opencv_traincascade","-data", "../haar_train/cascade", "-vec", "../haar_train/whale_face.vec",
#                               "-bg", "../haar_train/bg.txt", "-numStages", "20", "-minHitRate", "0.999", "-maxFalseAlarmRate",
#                               "0.5", "-numPos", "4850","-numNeg", "800", "-w", str(size), "-h", str(size), "-mode", "ALL", 
#                               "-precalcValBufSize", "1024","-precalcIdxBufSize", "1024"], stdout=subprocess.PIPE)
#    for c in iter(lambda: process.stdout.read(1), ''):
#        sys.stdout.write(c)
#        f.write(c)


Info file name: ../haar_train_1step/annotations_normalized_straight.lst
Img file name: (NULL)
Vec file name: ../haar_train_1step/whale_face_80_normalized_straight.vec
BG  file name: (NULL)
Num: 10000
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 40
Height: 80
Create training samples from images collection...
Done. Created 10000 samples



Parameters I change when creating the cascade files:
    - width and height of positive images
    - numStages in opencv_traincascade
    - minhitrate

In [ ]:
import subprocess
import numpy as np

totalPositiveSamples = 10000
Im_size = [40,60,80]
NumCascadeStages = [25,30]
MinHitRate = [0.99, 0.995, 0.999]

for i in Im_size:
    for j in NumCascadeStages:
        for k in MinHitRate:
            num = np.int(totalPositiveSamples / (1 + (j - 1) * (1 - k))*0.95)
            #print 'opencv_traincascade -data ../haar_train_1step/cascade_'+str(i)+'_'+str(j)+'_'+str(k)+' -vec ../haar_train_1step/whale_face_'+str(i)+'.vec -bg ../haar_train_1step/bg.txt -numStages '+str(j)+' -minHitRate '+str(k)+' -maxFalseAlarmRate 0.5 -numPos '+str(num)+' -numNeg 2000 -w '+str(i)+' -h '+str(i)+' -mode ALL -precalcValBufSize 1024 -precalcIdxBufSize 1024'
            subprocess.Popen('nohup opencv_traincascade -data cascade_'+str(i)+'_'+str(j)+'_'+str(k)+'_normalized_straight -vec whale_face_'+str(i)+'_normalized_straight.vec -bg bg.txt -numStages '+str(j)+' -minHitRate '+str(k)+' -maxFalseAlarmRate 0.5 -numPos '+str(num)+' -numNeg 4000 -w '+str(i/2)+' -h '+str(i)+' -featureType LBP -precalcValBufSize 5000 -precalcIdxBufSize 5000 > out_'+str(i)+'_'+str(j)+'_'+str(k)+'_normalized_straight.txt &', shell=True)
            subprocess.Popen('mkdir cascade_'+str(i)+'_'+str(j)+'_'+str(k)+'_normalized_straight', shell=True)

evaluate the cascades

In [15]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.misc import imsave
import os
from PIL import Image
from scipy.ndimage.interpolation import rotate
import glob
from scipy.misc import imresize

# based on previous code
mean = 111.031397463 
std = 18.4898562094

Im_size = [40,60,80]
NumCascadeStages = [25, 30]
MinHitRate = [0.99,0.995,0.999]

# delete all images in test directory
files = glob.glob('../haar_train_1step/test/*')
for f in files:
    os.remove(f)

files = []
for file in os.listdir("../imgs/"):
    if file.endswith('.jpg'):
        files.append(file[:-4])

# number of images to map 
n = 100

if n > len(files):
    n = len(files)

min_neighbors = np.arange(1,10,step = 1)
min_neighbors = np.append(min_neighbors,10e0**np.linspace(1e0,2e0,21e0))
rot = np.linspace(0,350,36)

scalefac = 1.01

for ii in range(n):
    f = np.random.choice(files)
    files.remove(f)
        
    if ii%10 == 0:
        print ii,' images done.'

    # open image
    im = np.array(Image.open('../imgs/'+f+'.jpg'))

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    gray = (gray - np.mean(gray)) / np.std(gray)
    # restandardize based on the average means and stds
    gray = (gray*std)+mean
    gray[gray < 0e0] = 0e0
    gray[gray > 255e0] = 255e0
    gray_ori = np.array(gray, dtype='uint8')
    
    # rotate image to find the head
    kk = 0
    found = np.zeros(len(rot))
    
    print f
    
    for i in Im_size:
        for j in NumCascadeStages:
            for k in MinHitRate:
                found_old = 0
                for jj in np.arange(len(rot)):
                    gray = rotate(gray_ori,rot[jj])

                    a = np.min(np.shape(gray))

                    face_cascade = cv2.CascadeClassifier('../haar_train_1step/cascade_'+str(i)+'_'+str(j)+'_'+str(k)+'_normalized_straight/cascade.xml')
                    
                    found = 0
                    
                    faces_all = []
                    
                    for l in np.arange(len(min_neighbors)):
                        faces = face_cascade.detectMultiScale(gray, scaleFactor=scalefac, minNeighbors = int(min_neighbors[l]), minSize=(a/20, a/20))
                        if np.shape(faces)[0] == 1:
                            found = found + 1
                            faces_all.append(faces)
                    if found > found_old:
                        face_cut = faces_all[0]

                        gray_copy = np.copy(gray)
                        for (x,y,w,h) in face_cut:
                            cv2.rectangle(gray_copy,(x,y),(x+w,y+h),(255,255,255),10)
                            imsave('../haar_train_1step/test2/'+f+'_'+str(i)+'_'+str(j)+'_'+str(k)+'_'+str(jj)+'.jpg', imresize(gray_copy,[np.shape(gray)[0]/4,np.shape(gray)[1]/4]))
                            #im_rot = rotate(im,rot[jj])
                            #imsave('../haar_train_1step/test2/head_'+f+'_'+str(i)+'_'+str(j)+'_'+str(k)+'_'+str(jj)+'.jpg', im_rot[y:y+h,x:x+w])
                    found_old = np.max([found,found_old])

[   0.   10.   20.   30.   40.   50.   60.   70.   80.   90.  100.  110.
  120.  130.  140.  150.  160.  170.  180.  190.  200.  210.  220.  230.
  240.  250.  260.  270.  280.  290.  300.  310.  320.  330.  340.  350.]


NameError: name 'stop' is not defined

check the score of different cascades

gradients around the central grid point:

60_20_0.99:  1 - 38/104 = 63%
60_20_0.995: 1 - 33/104 = 68%
60_20_0.999: 1 - 53/104 = 49%

60_15_0.995: 1 - 84/104 = 19%
60_20_0.995: 1 - 33/104 = 68%
60_25_0.995: 1 - 25/104 = 76%

40_20_0.995: 1 - 34/108 = 68%
60_20_0.995: 1 - 33/104 = 68%
80_20_0.995: 1 - 41/95 = 57%

prediction: 40_25_0.995 should also have a similarly high score to 60_25_0.995 but it is faster to run.
40_25_0.995: 1 - 27/105 = 74%

gradients centered around 60_25_0.995:

60_25_0.99:  1 - 25/104 = 76%
60_25_0.995: 1 - 25/104 = 76%
60_25_0.999: 1 - 25/104 = 76%

40_25_0.995: 1 - 27/105 = 74%
60_25_0.995: 1 - 25/104 = 76%
80_25_0.995: 1 - 23/93 = 75%

other checks to be on the safe side:
40_25_0.99: 1 - 30/106 = 72%

started runs of 60_30_0.995 and 60_35_0.995.

in the current vec files, 10 random rotation angles are used. I also check how the score changes if i use 20, 30, 40 angles instead (larger vec file).

I also studied other cascade parameters: number of rotation angle, scale factor, and number of min_neighbor values
the study is on 10 images only so the accuracy isn't good. but i still learnt from it and improved the score.

40 25 0.99
    20 deg rot angle
    1.05 scale factor
    31 min_neighbor values
        7 out of 10 images successfully identified
        
    10 deg rot angle
    1.05 scale factor
    31 min_neighbor values
        7 out of 10

    20 deg rot angle
    1.01 scale factor
    31 min_neighbor values
       8 out of 10!

    20 deg rot angle
    1.01 scale factor
    51 min_neighbor values
        7 out of 10

Score of new cascades
image size: 40, 60, 80
hit rates: 0.99, 0.995, 0.999
stages: 25, 30

40_25_0.99:  84%
40_25_0.995: 88%
40_25_0.999: 86%

40_30_0.999: 90%

60_25_0.99:  85%
60_25_0.999: 89%
60_25_0.995: 87%

80_25_0.995: 87%
80_25_0.999: 86%

find face in all images

In [128]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.misc import imsave
import os
from PIL import Image
from scipy.ndimage.interpolation import rotate
import glob
from scipy.misc import imresize
import csv
from collections import Counter

def read_data_csv(name):
   """
   read cvs file and return contents
   """

   file_name = []
   whale_ID = []
   
   with open(name, 'rb') as f:
      reader = csv.reader(f)

      # skip header
      reader.next()

      for row in reader:  
         file_name.append(row[0][:-4])
         whale_ID.append(row[1])
           
   file_name = np.array(file_name)
   whale_ID = np.array(whale_ID)
   
   return file_name,whale_ID
        
# mean and std of images, based on previous code
mean = 111.031397463 
std = 18.489856209

# cascade parameters
Im_size = 60
NumCascadeStages = 30
MinHitRate = 0.995

scalefac = 1.01
min_neighbors = np.arange(1,10,step = 1)
min_neighbors = np.append(min_neighbors,10e0**np.linspace(1e0,4e0,11e0))
min_neighbors = min_neighbors.astype(int)
rot = np.linspace(0,350,36)

# collecing the image file names
#files = []
#for file in os.listdir("../imgs/"):
#    if file.endswith('.jpg'):
#        files.append(file[:-4])

# collecting the labelled images
files,ID = read_data_csv('../train.csv')

# number of images to map 
n = 500
if n > len(files):
    n = len(files)

for ii in range(n):
    # check which images had been analyzed already
    whale_faces = []
    for file in os.listdir("../whale_faces/"):
        if file.endswith('.jpg'):
            whale_faces.append(file[:-4])

    # select a new file
    f = np.random.choice(list(set(files) - set(whale_faces)))
    
    if ii%10 == 0:
        print ii,' images done.'

    # open image
    im = np.array(Image.open('../imgs/'+f+'.jpg'))
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    # standardize and restandardize based on the average mean and std
    gray = (gray - np.mean(gray)) / np.std(gray)
    gray = (gray*std)+mean
    gray[gray < 0e0] = 0e0
    gray[gray > 255e0] = 255e0
    gray_ori = np.array(gray, dtype='uint8')
    
    # loop through rotation angles
    print f    
    max_min_neighbors_old = 0
    for jj in np.arange(len(rot)):
        # rotate image
        gray = rotate(gray_ori,rot[jj])        
        
        # load the cascade
        face_cascade = cv2.CascadeClassifier('../haar_train_1step/cascade_'+str(Im_size)+'_'+str(NumCascadeStages)+'_'+str(MinHitRate)+'_normalized_straight/cascade.xml')

        # increase min_neighbors and count how many times one face is found on the image 
        # (if min_neighbors is small, multiple rectangles are marked.)
        # if 0 face is found, end the while loop
        # save the face with the highest found value
        a = np.min(np.shape(gray_ori))        
        faces_all = []
        max_min_neighbors = 0
        l = 0
        nr_faces = 1
        while (nr_faces > 0) and (l < len(min_neighbors)):
            faces = face_cascade.detectMultiScale(gray, scaleFactor=scalefac, minNeighbors = min_neighbors[l], minSize=(a/30, a/30))
            nr_faces = np.shape(faces)[0]
            if nr_faces == 1:
                max_min_neighbors = min_neighbors[l]
                faces_all.append(faces)
            l = l + 1
        # save the face if its found value is higher than any previous encountered value
        print jj,max_min_neighbors
        if max_min_neighbors > max_min_neighbors_old:
            face_cut = faces_all[-1]
            #gray_copy = np.copy(gray)
            im_rot = rotate(im,rot[jj])
            for (x,y,w,h) in face_cut:
                #cv2.rectangle(gray_copy,(x,y),(x+w,y+h),(255,255,255),10)
                #imsave('../haar_train_1step/test3/'+f+'_'+str(jj)+'.jpg', imresize(gray_copy,[np.shape(gray)[0]/4,np.shape(gray)[1]/4]))
                imsave('../whale_faces/'+f+'.jpg', im_rot[y:y+h,x:x+w])
        max_min_neighbors_old = np.max([max_min_neighbors,max_min_neighbors_old])

0  images done.
w_6414
w_5536
w_3635
w_7704
w_7443
w_266
w_4919
w_614
w_1124
w_2379
